In [4]:
import numpy as np
import pandas as pd
import ast
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pickle import dump
from pinecone import Pinecone

c:\Users\kumar\Documents\ME\My Project\python\ML\Movie Recommandation\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
movie = pd.read_csv('movies_metadata.csv')
credits = pd.read_csv('credits.csv')

C:\Users\kumar\AppData\Local\Temp\ipykernel_16524\1214811965.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie = pd.read_csv('movies_metadata.csv')


In [6]:
# Convert the 'id' column in either movie or credits DataFrame to a consistent type before merging
movie['id'] = pd.to_numeric(movie['id'], errors='coerce')
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')

# Now perform the merge operation
new_movie = pd.merge(movie, credits, on='id')

In [7]:
new_movie = new_movie[['id','title','genres','overview','tagline','cast','crew']]

In [8]:
new_movie['title'].fillna(movie['original_title'], inplace=True)
new_movie.fillna("", inplace=True)

C:\Users\kumar\AppData\Local\Temp\ipykernel_16524\2684447749.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_movie['title'].fillna(movie['original_title'], inplace=True)


In [9]:
new_movie.isnull().sum()

id          0
title       0
genres      0
overview    0
tagline     0
cast        0
crew        0
dtype: int64

In [10]:
def convert_text(text, columnName=["name"], filter_by=None):
    result = []
    for i in ast.literal_eval(text):
        # Skip if any filter condition is not met
        if filter_by and not all(i.get(key) == value for key, value in filter_by.items()):
            continue

        # Process columns and append to the result
        result.extend(i[j].replace(" ", "") for j in columnName)

    return " ".join(result)

In [11]:
new_movie['genres'] = new_movie['genres'].apply(convert_text)
new_movie['cast'] = new_movie['cast'].apply(lambda x: convert_text(x, columnName=['name']))
new_movie['crew'] = new_movie['crew'].apply(lambda x: convert_text(x, columnName=['name'], filter_by={'job':'Director'}))

In [12]:
new_movie['tag'] = new_movie['title'] + " " + new_movie['overview'] + " " + new_movie['tagline'] + " " + new_movie['genres'] + " " + new_movie['cast'] + " " + new_movie['crew']
new_movie.drop(columns=['overview','tagline','genres','cast','crew'], inplace=True)

In [13]:
def stem_text(text):
    text = text.lower()
    text = text.replace(",", " ")
    ps = PorterStemmer()
    return " ".join([ps.stem(word) for word in text.split()])
new_movie['tag'] = new_movie['tag'].apply(stem_text)

In [96]:
new_movie.shape

(45538, 3)

In [15]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_movie['tag']).toarray()

In [16]:
vectors = vectors.astype(np.float32)

In [ ]:
pc = Pinecone(api_key="pcsk_28BYaV_271pDGPxzVmbyU4io4GriUoX7sYjNseGoYH7iZpUjgcSHXBoxdNKi8CKwCLgLvi")

index = pc.Index("movie-recommandation")
vectors_list = [{'id': str(x[0]), 'values': x[1].astype(np.float32).tolist()} for x in list(enumerate(vectors))]

In [72]:
def upsert_with_retry(index, vectors, max_retries=3, delay=5):
    for i in vectors:
        if np.any(i['values']):
            pass
        else:
            i['values'][0] = 0.01
            print(f"Skipping vector with ID '{i['id']}' because it contains only zeros.")
        for attempt in range(max_retries):
                try:
                    index.upsert(vectors=[i])
                    break
                except ProtocolError as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt < max_retries - 1:
                        time.sleep(delay)
                    else:
                        print(f"Failed to upsert vector with ID '{i['id']}' after {max_retries} attempts.")
                        

In [73]:
upsert_with_retry(index, vectors_list)

NameError: name 'ProtocolError' is not defined

In [1]:
def recommend_movie(index, title):
    Li = []
    movie_index = new_movie[new_movie['title'] == title].index[0]
    print(movie_index)
    movies_list = index.query(id=str(movie_index), top_k=6).matches
    for i in movies_list:
        Li.append(new_movie.iloc[int(i.id)].title)
    
    return Li

recommend_movie(index, 'The Avengers')

NameError: name 'index' is not defined